In [1]:
import numpy as np
import pandas as pd
from faker import Faker

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from datetime import datetime

In [2]:
fake = Faker('it_IT')

In [3]:
def timestamp(dt):
    epoch = datetime.utcfromtimestamp(0)
    return (dt - epoch.date()).total_seconds() * 1000.0

Si creano i vettori da usare per generare dati

In [4]:
# GENERATORS
SESSO = ["Donna", "Uomo", "Altro"]
ANNO = ['2020', '2021']
MESE = ["Gen", "Feb", "Mar", "Apr", "Mag", "Giu", "Lug", "Ago", "Set", "Ott", "Nov", "Dic"]
REGIONE = ["Sicilia", "Piemonte", "Marche", "Valle d'Aosta", "Toscana", "Campania", "Puglia", "Veneto",
           "Lombardia", "Emilia-Romagna", "Trentino-Alto Adige", "Sardegna", "Molise", "Calabria",
           "Abruzzo", "Lazio", "Liguria", "Friuli-Venezia Giulia", "Basilicata", "Umbria"]
PELLE = ["Molto chiara", "Chiara", "Intermedia", "Scura", "Molto scura"]
ISTRUZIONE = ["Primaria", "Secondaria", "Laurea Breve", "Master/PhD"]
ATTIVITA = ["Pesante", "Moderata", "Leggera"]
ABITUDINE = ["Nessuna", "Lieve", "Ex", "Moderata", "Intensa"]
DIETA = ["Poco equilibrata", "Regolare", "Molto equilibrata"]
TIMING = ["Meno di 1 anno fa", "1-3 anni fa", "3-5 anni fa", "Oltre 5 anni fa"]
SINTOMI = ["Lievi", "Moderati", "Intensi"]
VISITE = [
    "Agoaspirato", "Marcatori tumorali", "Colonscopia",
    "Visita odontoiatrica",
    "Ecografia mammaria", "Ecografia transvaginale", "Hpv Dna Test", "Pap-Test", "Mammografia",
    "Elettrocardiogramma", "Holter", "Coronarografia", "Ecocardiografia",
    "Biopsia prostatica",
    "Spirometria", "Esame del sangue",
    "Pet", "Risonanza magnetica (RM)", "Scintigrafia",
]
AREE = ["Oncologica", "Odontoiatrica", "Ginecologica",
        "Cardiologica", "Andrologica", "Generale", "Diagnostica"]

In [5]:
# DF
COLUMNS_USER = [
    "data",
    "nome_cognome",
    "data_iscrizione",
    "eta",
    "sesso",
    "regione",
    "peso",
    "altezza",
    "pelle",
    "istruzione",
    "quiz_categorieRischio",
    "quiz_covid",
    "quiz_familiarita",
    "quiz_salutePersonale",
    "quiz_stileDiVita",
    "quiz_vaccini",
    "diabete",
    "diag_diabete",
    "cardiopatia",
    "ipertensione",
    "attivita_lavoro",
    "attivita_tl",
    "fumo",
    "alcol",
    "sostanze",
    "dieta"
    "fam_onco",
    "fam_meta",
    "fam_cardio",
    "fam_resp",
    "fam_onco_gino",
    "fam_onco_andro"
    "vacc_pnmccc",
    "vacc_zoster",
    "vacc_influ"
    "cup_derma",
    "cup_sangue",
    "cup_odonto",
    "cup_glicemia",
    "cup_cole",
    "cup_cardio",
    "cup_oculo",
    "cup_spiro",
    "cup_feci",
    "cup_andro",
    "cup_colon",
    "cup_gino",
    "cup_seno",
    "cup_tac",
    "cup_onco",
    "covid",
    "sintomi",
    "vaccino"
]
COLUMNS_VISIT = [
    "Data",
    "Eseguita",
    "Nome",
    "Tipologia",
    "Prenotata",
]

In [6]:
# QUESTIONARI
ID = [
    "nome_cognome",
    "data_iscrizione",
    "eta",
    "sesso",
    "regione",
    "peso",
    "altezza",
    "pelle",
    "istruzione",
    "quiz_categorieRischio",
    "quiz_covid",
    "quiz_familiarita",
    "quiz_salutePersonale",
    "quiz_stileDiVita",
    "quiz_vaccini",
]
RISCHIO = [
    "diabete",
    "diag_diabete",
    "cardiopatia",
    "ipertensione",
]
STILE = [
    "attivita_lavoro",
    "attivita_tl",
    "fumo",
    "alcol",
    "sostanze",
    "dieta"
]
FAMILIARITA = [
    "fam_onco",
    "fam_meta",
    "fam_cardio",
    "fam_resp",
    "fam_onco_gino",
    "fam_onco_andro"
]
VACCINI = [
    "vacc_pnmccc",
    "vacc_zoster",
    "vacc_influ"
]
CUP = [
    "cup_derma",
    "cup_sangue",
    "cup_odonto",
    "cup_glicemia",
    "cup_cole",
    "cup_cardio",
    "cup_oculo",
    "cup_spiro",
    "cup_feci",
    "cup_andro",
    "cup_colon",
    "cup_gino",
    "cup_seno",
    "cup_tac",
    "cup_onco"
]
COVID = [
    "covid",
    "sintomi",
    "vaccino"
]
VISITA = [
    "Data",
    "Eseguita",
    "Nome",
    "Tipologia",
    "Prenotata",
]

In [7]:
def create_users():
    my_row = {}
    my_visit = {}
    # IDENTIFICAZIONE
    my_row['data'] = fake.date_between_dates(date_start=datetime(2020, 1, 1))
    my_row['data_iscrizione'] = timestamp(dt=my_row['data'])
    my_row['sesso'] = np.random.choice(SESSO, 1, p=[0.45, 0.45, 0.1])[0]
    if my_row['sesso'] == 'Donna':
        my_row['nome_cognome'] = fake.name_female()
    elif my_row['sesso'] == 'Uomo':
        my_row['nome_cognome'] = fake.name_male()
    else:
        my_row['nome_cognome'] = fake.name()
    my_row['mail'] = my_row['nome_cognome'].lower().replace(' ', '.') + '@gmail.com'
    my_row['eta'] = fake.pyint(min_value=18, max_value=90, step=1)
    my_row['regione'] = np.random.choice(REGIONE, 1)[0]
    my_row['peso'] = fake.pyint(min_value=40, max_value=140, step=1)
    if my_row['peso'] > 90:
        my_row['altezza'] = fake.pyint(min_value=160, max_value=205, step=1)
    else:
        my_row['altezza'] = fake.pyint(min_value=140, max_value=180, step=1)
    my_row['pelle'] = np.random.choice(PELLE, 1)[0]
    if my_row['eta'] < 26:
        my_row['istruzione'] = np.random.choice(ISTRUZIONE[:-1], 1)[0]
    else:
        my_row['istruzione'] = np.random.choice(ISTRUZIONE, 1)[0]
    
    # CATEGORIA A RISCHIO
    my_row['diabete'] = fake.pybool()
    if my_row['diabete']:
        my_row['diag_diabete'] = fake.date().split('-')[0]
    my_row['cardiopatia'] = fake.pybool()
    my_row['ipertensione'] = fake.pybool()
    
    # STILE  DI VITA
    my_row['attivita_lavoro'] = np.random.choice(ATTIVITA, 1)[0]
    my_row['attivita_tl'] = np.random.choice(ATTIVITA, 1)[0]
    my_row['fumo'] = np.random.choice(ABITUDINE, 1)[0]
    my_row['alcol'] = np.random.choice(ABITUDINE, 1)[0]
    my_row['sostanze'] = np.random.choice(ABITUDINE, 1)[0]
    my_row['dieta'] = np.random.choice(DIETA, 1)[0]
    
    # FAMILIARITA
    my_row['fam_onco'] = fake.pybool()
    my_row['fam_meta'] = fake.pybool()
    my_row['fam_cardio'] = fake.pybool()
    my_row['fam_resp'] = fake.pybool()
    if my_row['sesso'] == 'Donna' or my_row['sesso'] == 'Altro':
        my_row['fam_onco_gino'] = fake.pybool()
    if my_row['sesso'] == 'Uomo' or my_row['sesso'] == 'Altro':
        my_row['fam_onco_andro'] = fake.pybool()
    
    # VACCINI
    if my_row['eta'] >= 65:
        my_row['vacc_pnmccc'] = fake.pybool()
        my_row['vacc_zoster'] = fake.pybool()
        my_row['vacc_influ'] = fake.pybool()
    
    # ULTIMO CHECK-UP
    if my_row['pelle'] in PELLE[0:3]:
        my_row['cup_derma'] = np.random.choice(TIMING, 1)[0]
    my_row['cup_sangue'] = np.random.choice(TIMING, 1)[0]
    my_row['cup_odonto'] = np.random.choice(TIMING, 1)[0]
    if my_row['eta'] >= 25:
        my_row['cup_cardio'] = np.random.choice(TIMING, 1)[0]
        my_row['cup_oculo'] = np.random.choice(TIMING, 1)[0]
        if my_row['dieta'] == DIETA[0] or my_row['fam_meta']:
            my_row['cup_glicemia'] = np.random.choice(TIMING, 1)[0]
            if my_row['attivita_tl'] == ATTIVITA[2]:
                my_row['cup_cole'] = np.random.choice(TIMING, 1)[0]
    if my_row['sesso'] == 'Donna' or my_row['sesso'] == 'Altro':
        my_row['cup_gino'] = np.random.choice(TIMING, 1)[0]
        if my_row['eta'] >= 30:
            my_row['cup_seno'] = np.random.choice(TIMING, 1)[0]
    if my_row['sesso'] == 'Uomo' or my_row['sesso'] == 'Altro':
        my_row['cup_andro'] = np.random.choice(TIMING, 1)[0]
    if my_row['eta'] >= 40:
        my_row['cup_feci'] = np.random.choice(TIMING, 1)[0]
    if my_row['eta'] >= 50:
        my_row['cup_colon'] = np.random.choice(TIMING, 1)[0]
    if my_row['fam_onco']:
        my_row['cup_onco'] = np.random.choice(TIMING, 1)[0]
        if my_row['alcol'] in ABITUDINE[2:] or my_row['fumo'] in ABITUDINE[2:]:
            my_row['cup_tac'] = np.random.choice(TIMING, 1)[0]
    
    #COVID
    my_row['covid'] = fake.pybool()
    if my_row['covid']:
        my_row['sintomi'] = np.random.choice(SINTOMI, 1)[0]
    my_row['vaccino'] = fake.pybool()
    
    if bool(set(my_row.keys()) & set(RISCHIO)):
        my_row['quiz_categorieRischio'] = True
    if bool(set(my_row.keys()) & set(FAMILIARITA)):
        my_row['quiz_familiarita'] = True
    if bool(set(my_row.keys()) & set(COVID)):
        my_row['quiz_covid'] = True
    if bool(set(my_row.keys()) & set(VACCINI)):
        my_row['quiz_vaccini'] = True
    if bool(set(my_row.keys()) & set(STILE)):
        my_row['quiz_stileDiVita'] = True
    if bool(set(my_row.keys()) & set(CUP)):
        my_row['quiz_salutePersonale'] = True
    
    my_visit['mail'] = my_row['mail']
    my_visit['Data'] = dt=fake.date_between_dates(date_start=my_row['data'])
    my_visit['Nome'] = np.random.choice(VISITE, 1)[0]
    if my_visit['Nome'] in VISITE[0:3]:
        my_visit['Tipologia'] = "Oncologica"
    elif my_visit['Nome'] == 'Visita odontoiatrica':
        my_visit['Tipologia'] = "Odontoiatrica"
    elif my_visit['Nome'] in VISITE[4:9]:
        my_visit['Tipologia'] = "Ginecologica"
    elif my_visit['Nome'] in VISITE[9:13]:
        my_visit['Tipologia'] = "Cardiologica"
    elif my_visit['Nome'] == 'Biopsia prostatica':
        my_visit['Tipologia'] = "Andrologica"
    elif my_visit['Nome'] in VISITE[14:16]:
        my_visit['Tipologia'] = "Generale"
    elif my_visit['Nome'] in VISITE[16:]:
        my_visit['Tipologia'] = "Diagnostica"
    my_visit['Prenotata'] = fake.pybool()
    my_visit['Eseguita'] = fake.pybool()
    
    gen_u = (k for k in COLUMNS_USER if k not in my_row.keys())
    for k in gen_u:
        my_row[k] = None
    
    gen_v = (k for k in COLUMNS_VISIT if k not in my_visit.keys())
    for k in gen_v:
        my_visit[k] = None
    
    return my_row, my_visit

In [8]:
output_users = []
output_visits = []
for i in range(1000):
    my_user_row, my_visit_row = create_users()
    output_users.append(my_user_row)
    output_visits.append(my_visit_row)

In [9]:
output_users_df = pd.DataFrame(output_users)
output_visits_df = pd.DataFrame(output_visits)

In [10]:
output_users_df.head()

,data,data_iscrizione,sesso,nome_cognome,mail,eta,regione,peso,altezza,pelle,...,cup_feci,cup_colon,cup_gino,cup_seno,cup_tac,cup_onco,sintomi,vacc_pnmccc,vacc_influ,cup_derma
0,2020-08-02,1.596326e+12,Uomo,Dott. Emilio Praga,dott..emilio.praga@gmail.com,19,Marche,86,178,Scura,...,None,None,None,None,None,None,None,NaN,NaN,NaN
1,2021-04-07,1.617754e+12,Donna,Liliana Aldobrandi,liliana.aldobrandi@gmail.com,38,Veneto,127,161,Scura,...,None,None,Oltre 5 anni fa,Meno di 1 anno fa,None,None,Moderati,NaN,NaN,NaN
2,2021-01-07,1.609978e+12,Donna,Milena Tarantini,milena.tarantini@gmail.com,59,Veneto,40,180,Scura,...,Oltre 5 anni fa,Meno di 1 anno fa,Meno di 1 anno fa,3-5 anni fa,1-3 anni fa,Meno di 1 anno fa,Moderati,NaN,NaN,NaN
3,2020-08-01,1.596240e+12,Altro,Carmelo Bompiani,carmelo.bompiani@gmail.com,83,Marche,122,180,Chiara,...,Oltre 5 anni fa,3-5 anni fa,1-3 anni fa,3-5 anni fa,None,None,Lievi,False,False,3-5 anni fa
4,2020-01-11,1.578701e+12,Altro,Tonino Pozzecco,tonino.pozzecco@gmail.com,79,Calabria,41,161,Intermedia,...,1-3 anni fa,3-5 anni fa,Oltre 5 anni fa,Oltre 5 anni fa,None,1-3 anni fa,None,False,False,1-3 anni fa


In [11]:
output_visits_df.head()

,mail,Data,Nome,Tipologia,Prenotata,Eseguita
0,dott..emilio.praga@gmail.com,2021-01-21,Elettrocardiogramma,Cardiologica,True,False
1,liliana.aldobrandi@gmail.com,2021-04-09,Ecocardiografia,Cardiologica,False,False
2,milena.tarantini@gmail.com,2021-04-09,Risonanza magnetica (RM),Diagnostica,False,True
3,carmelo.bompiani@gmail.com,2021-01-04,Hpv Dna Test,Ginecologica,True,False
4,tonino.pozzecco@gmail.com,2020-02-27,Risonanza magnetica (RM),Diagnostica,True,False


In [12]:
output_users_df.to_csv(f'data/users_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}.csv')
output_visits_df.to_csv(f'data/visits_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}.csv')